# Wired Elements: experimenting Web Components

https://github.com/rough-stuff/wired-elements

## Libraries

In [1]:
from httpserver import start, stop
from http.server import SimpleHTTPRequestHandler, HTTPServer
import os
from urllib.parse import urlparse, parse_qs
import ipywidgets as widgets
import time
from IPython.display import HTML

## Backend: the HTTP server

### Widgets python implementation

In [2]:
logsw=widgets.Textarea(layout=widgets.Layout(width='50%'))
logsw.value = ""
bag ={}
class myHTTPRequestHandler(SimpleHTTPRequestHandler):
    callback={}
    def __init__(self, *args, directory=None,bag=bag, **kwargs):
        self.bag = bag
        self.directory = os.path.join(os.getcwd(),"widgets")
        super().__init__(*args, directory=self.directory, **kwargs)
        # print(self.directory)
    def end_headers(self):
        super().end_headers()
    def do_GET(self):
        self.parsed_path = urlparse(self.path)
        self.queryparams = parse_qs(self.parsed_path.query)
        if self.path.endswith('/version'):
            self.version()
        elif self.parsed_path.path.endswith('/setvalue'):
            self.setvalue()
        else:
            super().do_GET()
    def version(self):
        ans = '{"version": "0.0"}'
        eans = ans.encode()        
        self.send_response(200)
        self.send_header("Content-type", "application/json")
        self.send_header("Content-Length",len(eans))
        self.end_headers()
        self.wfile.write(eans)
    def setvalue(self):
        self.bag.update({self.queryparams['variable'][0]:self.queryparams['value'][0]})
        if self.queryparams['variable'][0] in myHTTPRequestHandler.callback:
            self.callback[self.queryparams['variable'][0]](self.queryparams['value'][0])
        self.version() 
    def log_message(self, format, *args):
        global logsw
        v = format % args
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        logsw.value += current_time + " " + v +"\n"
logsw

Textarea(value='', layout=Layout(width='50%'))

In [3]:
start(handler_class=myHTTPRequestHandler, port=8085)

## Frontend

### Widget javascript implementation: Using Web Components

In [4]:
class LoadModule(object):
    """This class is loading modules"""
    def _repr_javascript_(self):
        return '''
debugger;
const modulename = 'wired-elements';
let module = document.getElementById('modulename'+'.js');
if (module === null){
    let css_urls = [{"url": 'https://fonts.googleapis.com/css?family=Gloria+Hallelujah&display=swap', "rel": "stylesheet"},
                        {"url": "/proxy/8085/wired-elements/style.css", "rel": "stylesheet", "type": "text/css"}
                       ];
    let js_urls = [{"url": 'https://unpkg.com/wired-elements?module', "id": modulename+'.js'}
                  ];
    for (const css_url in css_urls){
        let fileref = document.createElement('link');
        fileref.rel = css_urls[css_url].rel;
        if ('type' in css_urls[css_url]){
            fileref.type = css_urls[css_url].type;
        }
        fileref.href = css_urls[css_url].url;
        document.head.append(fileref);
    }
    for (const js_url in js_urls){
        let script_= document.createElement('script');
        script_.src = js_urls[js_url].url;
        script_.type = "module"
        script_.id = js_urls[js_url].id
        document.head.appendChild(script_);
    }
}
'''

Starting httpd...



In [5]:
LoadModule()

In [6]:
def cb_monnom(value):
    global monnom
    monnom.value = value
myHTTPRequestHandler.callback.update({'monnom': cb_monnom})
wired = HTML("""
<main>
  <wired-card elevation="5">
    <h1>wired-elements demo</h1>
  </wired-card>
  <section>
    <wired-input placeholder="your name"></wired-input>
    <wired-button elevation="2">Submit</wired-button>
  </section>
</main>
""")
monnom = widgets.Label()
display(wired,monnom)

Label(value='')

In [7]:
class ExecuteModule(object):
    """This class execute the code once the module is loaded"""
    def _repr_javascript_(self):
        return '''debugger;
let script_= document.createElement('script');
script_.src = "/proxy/8085/wired-elements/main.js";
script_.type = "module"
document.head.appendChild(script_);
'''

In [8]:
ExecuteModule()

In [9]:
#stop()